# Chapter 10-2: 사용자 정의 예외

## 학습 목표
- 사용자 정의 예외 클래스 만들기
- 예외 계층 구조 설계하기
- raise 문으로 예외 발생시키기
- 실제 프로젝트에서의 예외 활용하기

## 1. 사용자 정의 예외 기초

내장 예외만으로는 모든 상황을 표현하기 어려울 때 사용자 정의 예외를 만들 수 있습니다.

### 기본 원칙
- Exception 클래스를 상속받아 생성
- 명확하고 설명적인 이름 사용
- 적절한 에러 메시지 제공
- 계층 구조로 조직화

In [ ]:
print("=== 기본 사용자 정의 예외 ===")

# 1. 가장 간단한 형태
class CustomError(Exception):
    """사용자 정의 예외 기본 형태"""
    pass

# 2. 메시지를 포함하는 예외
class ValidationError(Exception):
    """데이터 검증 실패 예외"""
    def __init__(self, message, field_name=None):
        super().__init__(message)
        self.message = message
        self.field_name = field_name
    
    def __str__(self):
        if self.field_name:
            return f"Validation error in '{self.field_name}': {self.message}"
        return f"Validation error: {self.message}"

# 3. 추가 정보를 포함하는 예외
class BusinessLogicError(Exception):
    """비즈니스 로직 오류 예외"""
    def __init__(self, message, error_code=None, details=None):
        super().__init__(message)
        self.message = message
        self.error_code = error_code
        self.details = details or {}
    
    def __str__(self):
        if self.error_code:
            return f"[{self.error_code}] {self.message}"
        return self.message
    
    def get_details(self):
        return self.details

# 테스트
def test_custom_exceptions():
    """사용자 정의 예외 테스트"""
    
    # CustomError 테스트
    try:
        raise CustomError("이것은 사용자 정의 예외입니다.")
    except CustomError as e:
        print(f"1. CustomError 포착: {e}")
    
    # ValidationError 테스트
    try:
        raise ValidationError("이메일 형식이 올바르지 않습니다.", "email")
    except ValidationError as e:
        print(f"2. ValidationError 포착: {e}")
        print(f"   필드명: {e.field_name}")
    
    # BusinessLogicError 테스트
    try:
        raise BusinessLogicError(
            "잔액이 부족합니다.", 
            "INSUFFICIENT_BALANCE",
            {"current_balance": 1000, "required_amount": 1500}
        )
    except BusinessLogicError as e:
        print(f"3. BusinessLogicError 포착: {e}")
        print(f"   오류 코드: {e.error_code}")
        print(f"   상세 정보: {e.get_details()}")

test_custom_exceptions()

## 2. 예외 계층 구조 설계

In [ ]:
print("=== 계층적 예외 구조 ===")

# 1. 최상위 애플리케이션 예외
class AppError(Exception):
    """애플리케이션 기본 예외"""
    def __init__(self, message, error_code=None):
        super().__init__(message)
        self.message = message
        self.error_code = error_code

# 2. 사용자 관련 예외들
class UserError(AppError):
    """사용자 관련 예외 기본 클래스"""
    pass

class UserNotFoundError(UserError):
    """사용자를 찾을 수 없음"""
    def __init__(self, user_id):
        super().__init__(f"사용자를 찾을 수 없습니다: {user_id}", "USER_NOT_FOUND")
        self.user_id = user_id

class InvalidCredentialsError(UserError):
    """잘못된 인증 정보"""
    def __init__(self, username=None):
        message = f"잘못된 인증 정보입니다."
        if username:
            message += f" (사용자: {username})"
        super().__init__(message, "INVALID_CREDENTIALS")
        self.username = username

class AccountLockedError(UserError):
    """계정 잠금"""
    def __init__(self, username, lock_reason=None):
        message = f"계정이 잠겨있습니다: {username}"
        if lock_reason:
            message += f" (사유: {lock_reason})"
        super().__init__(message, "ACCOUNT_LOCKED")
        self.username = username
        self.lock_reason = lock_reason

# 3. 데이터 관련 예외들
class DataError(AppError):
    """데이터 관련 예외 기본 클래스"""
    pass

class DataValidationError(DataError):
    """데이터 검증 실패"""
    def __init__(self, field_name, value, reason):
        message = f"데이터 검증 실패 - {field_name}: {reason}"
        super().__init__(message, "DATA_VALIDATION_FAILED")
        self.field_name = field_name
        self.value = value
        self.reason = reason

class DataNotFoundError(DataError):
    """데이터를 찾을 수 없음"""
    def __init__(self, data_type, identifier):
        message = f"{data_type}을(를) 찾을 수 없습니다: {identifier}"
        super().__init__(message, "DATA_NOT_FOUND")
        self.data_type = data_type
        self.identifier = identifier

# 4. 비즈니스 로직 예외들
class BusinessError(AppError):
    """비즈니스 로직 예외 기본 클래스"""
    pass

class InsufficientFundsError(BusinessError):
    """잔액 부족"""
    def __init__(self, current_balance, required_amount):
        shortage = required_amount - current_balance
        message = f"잔액이 부족합니다. 현재: {current_balance:,}원, 필요: {required_amount:,}원 (부족: {shortage:,}원)"
        super().__init__(message, "INSUFFICIENT_FUNDS")
        self.current_balance = current_balance
        self.required_amount = required_amount
        self.shortage = shortage

class OperationNotAllowedError(BusinessError):
    """허용되지 않은 작업"""
    def __init__(self, operation, reason):
        message = f"작업이 허용되지 않습니다: {operation} (사유: {reason})"
        super().__init__(message, "OPERATION_NOT_ALLOWED")
        self.operation = operation
        self.reason = reason

# 예외 계층 구조 테스트
def test_exception_hierarchy():
    """예외 계층 구조 테스트"""
    
    exceptions_to_test = [
        UserNotFoundError("user123"),
        InvalidCredentialsError("john_doe"),
        AccountLockedError("jane_doe", "너무 많은 로그인 시도"),
        DataValidationError("age", -5, "나이는 양수여야 합니다"),
        DataNotFoundError("상품", "PROD001"),
        InsufficientFundsError(10000, 15000),
        OperationNotAllowedError("계좌 삭제", "미결제 대출이 있음")
    ]
    
    for exc in exceptions_to_test:
        try:
            raise exc
        except AppError as e:
            print(f"\n🚨 {type(e).__name__}")
            print(f"   메시지: {e.message}")
            print(f"   오류 코드: {e.error_code}")
            
            # 각 예외별 특별한 속성 출력
            if isinstance(e, UserNotFoundError):
                print(f"   사용자 ID: {e.user_id}")
            elif isinstance(e, InsufficientFundsError):
                print(f"   부족 금액: {e.shortage:,}원")
            elif isinstance(e, DataValidationError):
                print(f"   문제 필드: {e.field_name}, 값: {e.value}")

test_exception_hierarchy()

## 3. 예외 처리 전략과 패턴

In [ ]:
print("=== 예외 처리 전략 ===")

import functools
import time
from datetime import datetime

# 1. 재시도 데코레이터
def retry(max_attempts=3, delay=1, exceptions=(Exception,)):
    """재시도 데코레이터"""
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            
            for attempt in range(max_attempts):
                try:
                    return func(*args, **kwargs)
                except exceptions as e:
                    last_exception = e
                    if attempt < max_attempts - 1:
                        print(f"   시도 {attempt + 1} 실패: {e}. {delay}초 후 재시도...")
                        time.sleep(delay)
                    else:
                        print(f"   모든 시도 실패. 최종 오류: {e}")
            
            raise last_exception
        return wrapper
    return decorator

# 2. 예외 변환 데코레이터
def convert_exceptions(exception_map):
    """예외 변환 데코레이터"""
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except Exception as e:
                for source_exc, target_exc in exception_map.items():
                    if isinstance(e, source_exc):
                        if callable(target_exc):
                            raise target_exc(str(e)) from e
                        else:
                            raise target_exc from e
                raise  # 매핑되지 않은 예외는 그대로 전파
        return wrapper
    return decorator

# 3. 실패하기 쉬운 함수들 (테스트용)
import random

@retry(max_attempts=3, delay=0.5, exceptions=(ConnectionError,))
def unreliable_network_call():
    """불안정한 네트워크 호출 시뮬레이션"""
    if random.random() < 0.7:  # 70% 확률로 실패
        raise ConnectionError("네트워크 연결 실패")
    return "네트워크 호출 성공!"

@convert_exceptions({
    ValueError: lambda msg: DataValidationError("unknown", "invalid", msg),
    KeyError: DataNotFoundError("키", "unknown")
})
def data_processing_function(data):
    """데이터 처리 함수"""
    if not isinstance(data, dict):
        raise ValueError("데이터는 딕셔너리여야 합니다")
    
    required_key = "required_field"
    if required_key not in data:
        raise KeyError(required_key)
    
    return f"처리 완료: {data[required_key]}"

# 테스트
print("1. 재시도 패턴 테스트:")
try:
    result = unreliable_network_call()
    print(f"   성공: {result}")
except ConnectionError as e:
    print(f"   최종 실패: {e}")

print("\n2. 예외 변환 패턴 테스트:")
test_data_cases = [
    {"required_field": "valid_data"},  # 성공
    "not_a_dict",  # ValueError -> DataValidationError
    {"wrong_field": "data"}  # KeyError -> DataNotFoundError
]

for i, test_data in enumerate(test_data_cases, 1):
    try:
        result = data_processing_function(test_data)
        print(f"   케이스 {i} 성공: {result}")
    except AppError as e:
        print(f"   케이스 {i} 실패: {type(e).__name__} - {e.message}")
    except Exception as e:
        print(f"   케이스 {i} 예상치 못한 오류: {type(e).__name__} - {e}")

## 4. 실전 예제: 은행 시스템

In [ ]:
print("=== 은행 시스템 예외 처리 ===")

# 은행 시스템 전용 예외들
class BankError(Exception):
    """은행 시스템 기본 예외"""
    pass

class AccountError(BankError):
    """계좌 관련 예외"""
    pass

class AccountNotFoundError(AccountError):
    """계좌를 찾을 수 없음"""
    def __init__(self, account_number):
        super().__init__(f"계좌를 찾을 수 없습니다: {account_number}")
        self.account_number = account_number

class InsufficientBalanceError(AccountError):
    """잔액 부족"""
    def __init__(self, current_balance, requested_amount):
        super().__init__(f"잔액 부족 - 현재: {current_balance:,}원, 요청: {requested_amount:,}원")
        self.current_balance = current_balance
        self.requested_amount = requested_amount

class AccountFrozenError(AccountError):
    """계좌 동결"""
    def __init__(self, account_number, reason):
        super().__init__(f"계좌가 동결되었습니다: {account_number} (사유: {reason})")
        self.account_number = account_number
        self.reason = reason

class TransactionError(BankError):
    """거래 관련 예외"""
    pass

class InvalidAmountError(TransactionError):
    """잘못된 금액"""
    def __init__(self, amount, reason):
        super().__init__(f"잘못된 금액: {amount} ({reason})")
        self.amount = amount
        self.reason = reason

class DailyLimitExceededError(TransactionError):
    """일일 한도 초과"""
    def __init__(self, daily_limit, current_usage, requested_amount):
        remaining = daily_limit - current_usage
        super().__init__(
            f"일일 한도 초과 - 한도: {daily_limit:,}원, "
            f"사용: {current_usage:,}원, 잔여: {remaining:,}원, "
            f"요청: {requested_amount:,}원"
        )
        self.daily_limit = daily_limit
        self.current_usage = current_usage
        self.requested_amount = requested_amount
        self.remaining = remaining

# 은행 계좌 클래스
class BankAccount:
    """은행 계좌 클래스"""
    
    def __init__(self, account_number, initial_balance=0, daily_limit=1000000):
        self.account_number = account_number
        self.balance = initial_balance
        self.daily_limit = daily_limit
        self.daily_usage = 0
        self.is_frozen = False
        self.freeze_reason = None
        self.transaction_history = []
    
    def _validate_amount(self, amount):
        """금액 유효성 검사"""
        if not isinstance(amount, (int, float)):
            raise InvalidAmountError(amount, "숫자가 아닙니다")
        
        if amount <= 0:
            raise InvalidAmountError(amount, "양수여야 합니다")
        
        if amount != int(amount):
            raise InvalidAmountError(amount, "원 단위여야 합니다")
    
    def _check_frozen(self):
        """계좌 동결 상태 확인"""
        if self.is_frozen:
            raise AccountFrozenError(self.account_number, self.freeze_reason)
    
    def _check_daily_limit(self, amount):
        """일일 한도 확인"""
        if self.daily_usage + amount > self.daily_limit:
            raise DailyLimitExceededError(
                self.daily_limit, self.daily_usage, amount
            )
    
    def _record_transaction(self, transaction_type, amount, description=""):
        """거래 기록"""
        transaction = {
            'timestamp': datetime.now().isoformat(),
            'type': transaction_type,
            'amount': amount,
            'balance_after': self.balance,
            'description': description
        }
        self.transaction_history.append(transaction)
    
    def deposit(self, amount, description="입금"):
        """입금"""
        self._validate_amount(amount)
        self._check_frozen()
        
        amount = int(amount)
        self.balance += amount
        self._record_transaction('DEPOSIT', amount, description)
        
        return self.balance
    
    def withdraw(self, amount, description="출금"):
        """출금"""
        self._validate_amount(amount)
        self._check_frozen()
        
        amount = int(amount)
        
        # 잔액 확인
        if self.balance < amount:
            raise InsufficientBalanceError(self.balance, amount)
        
        # 일일 한도 확인
        self._check_daily_limit(amount)
        
        self.balance -= amount
        self.daily_usage += amount
        self._record_transaction('WITHDRAWAL', amount, description)
        
        return self.balance
    
    def transfer(self, target_account, amount, description="계좌이체"):
        """계좌이체"""
        if not isinstance(target_account, BankAccount):
            raise AccountNotFoundError("유효하지 않은 계좌")
        
        # 출금 먼저 시도
        self.withdraw(amount, f"이체 출금 -> {target_account.account_number}")
        
        try:
            # 입금 시도
            target_account.deposit(amount, f"이체 입금 <- {self.account_number}")
        except Exception as e:
            # 입금 실패시 출금 취소 (보상 트랜잭션)
            self.deposit(amount, "이체 실패로 인한 복구")
            raise TransactionError(f"이체 실패: {e}") from e
        
        return True
    
    def freeze(self, reason):
        """계좌 동결"""
        self.is_frozen = True
        self.freeze_reason = reason
        self._record_transaction('FREEZE', 0, f"계좌 동결: {reason}")
    
    def unfreeze(self):
        """계좌 동결 해제"""
        self.is_frozen = False
        self.freeze_reason = None
        self._record_transaction('UNFREEZE', 0, "계좌 동결 해제")
    
    def get_balance(self):
        """잔액 조회"""
        self._check_frozen()
        return self.balance
    
    def __str__(self):
        status = "동결" if self.is_frozen else "정상"
        return f"계좌 {self.account_number}: {self.balance:,}원 ({status})"

# 은행 시스템 테스트
def test_bank_system():
    """은행 시스템 테스트"""
    
    # 계좌 생성
    account1 = BankAccount("123-456-789", 100000, 500000)
    account2 = BankAccount("987-654-321", 50000, 500000)
    
    print(f"초기 상태:")
    print(f"  {account1}")
    print(f"  {account2}")
    
    # 다양한 거래 시나리오 테스트
    test_scenarios = [
        ("정상 입금", lambda: account1.deposit(50000)),
        ("정상 출금", lambda: account1.withdraw(30000)),
        ("잔액 부족 출금", lambda: account1.withdraw(200000)),
        ("잘못된 금액 (음수)", lambda: account1.deposit(-1000)),
        ("잘못된 금액 (소수)", lambda: account1.withdraw(1000.5)),
        ("계좌 동결 후 출금", lambda: (account2.freeze("의심 거래"), account2.withdraw(10000))[1]),
        ("정상 계좌이체", lambda: account1.transfer(account2, 20000)),
        ("일일 한도 초과", lambda: account1.withdraw(600000)),
    ]
    
    print(f"\n=== 거래 시나리오 테스트 ===")
    for scenario_name, action in test_scenarios:
        try:
            if scenario_name == "계좌 동결 후 출금":
                account2.freeze("의심 거래")
                account2.withdraw(10000)
            else:
                result = action()
            print(f"✅ {scenario_name}: 성공")
        except BankError as e:
            print(f"❌ {scenario_name}: {type(e).__name__} - {e}")
        except Exception as e:
            print(f"⚠️  {scenario_name}: 예상치 못한 오류 - {type(e).__name__}: {e}")
    
    # 계좌 동결 해제
    account2.unfreeze()
    
    print(f"\n최종 상태:")
    print(f"  {account1}")
    print(f"  {account2}")
    
    # 거래 내역 출력
    print(f"\n{account1.account_number} 거래 내역:")
    for transaction in account1.transaction_history[-5:]:  # 최근 5개만
        print(f"  {transaction['timestamp'][:19]} | {transaction['type']} | {transaction['amount']:,}원 | 잔액: {transaction['balance_after']:,}원")

test_bank_system()

## 5. 정리 및 요약

### 🎯 학습한 내용

1. **사용자 정의 예외 기초**
   - Exception 클래스 상속
   - 생성자와 속성 정의
   - 의미있는 에러 메시지

2. **예외 계층 구조**
   - 논리적 그룹화
   - 상속 관계 설계
   - 카테고리별 예외 분류

3. **고급 예외 처리 패턴**
   - 재시도 메커니즘
   - 예외 변환
   - 데코레이터 활용

4. **실전 적용**
   - 은행 시스템 예제
   - 비즈니스 로직과 예외 통합
   - 보상 트랜잭션 패턴

### 💡 설계 원칙

1. **명확성**: 예외 이름과 메시지가 문제를 명확히 설명
2. **계층성**: 논리적 그룹화와 상속 관계
3. **정보성**: 디버깅에 필요한 추가 정보 포함
4. **일관성**: 프로젝트 전체에서 일관된 예외 처리
5. **복구 가능성**: 예외 상황에 대한 적절한 대응 방안

### 🚀 활용 분야

- **웹 애플리케이션**: HTTP 상태코드와 연동
- **API 서버**: 클라이언트 친화적 에러 응답
- **데이터 처리**: 검증 및 변환 오류 처리
- **비즈니스 로직**: 도메인 특화 예외
- **시스템 통합**: 외부 서비스 오류 처리

다음 장에서는 모든 지식을 종합한 **최종 프로젝트**를 진행하겠습니다!